# Getting Started with AI on Jetson Nano
### Interactive Classification Tool

"Getting Started with AI on Jetson Nano"


### Camera
First, create your camera and set it to `running`.  Uncomment the appropriate camera selection lines, depending on which type of camera you're using (USB or CSI). This cell may take several seconds to execute.

<div style="border:2px solid black; background-color:#e3ffb3; font-size:12px; padding:8px; margin-top: auto;">
    <h4><i>Tip</i></h4>
    <p>There can only be one instance of CSICamera or USBCamera at a time.  Before starting this notebook, make sure you have executed the final "shutdown" cell in any other notebooks you have run so that the camera is released.
    </p>
</div>

In [ ]:
# Check device number
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 0 Jun  1 05:45 /dev/video0


In [ ]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera

# # for USB Camera (Logitech C270 webcam), uncomment the following line
# camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

# for CSI Camera (Raspberry Pi Camera Module V2), uncomment the following line
camera = CSICamera(width=224, height=224, capture_device=0) # confirm the capture_device number

camera.running = True
print("camera created")

camera created


### Task
Next, define your project `TASK` and what `CATEGORIES` of data you will collect.  You may optionally define space for multiple `DATASETS` with names of your choosing.

Uncomment/edit the associated lines for the classification task you're building and execute the cell.
This cell should only take a few seconds to execute.

In [ ]:
# Copyright 2020 NVIDIA
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import torch
import torch.utils.data
import glob
import PIL.Image
import subprocess
import cv2
import os
import uuid
import subprocess


class ImageClassificationDataset(torch.utils.data.Dataset):

    def __init__(self, directory, categories, transform=None):
        self.categories = categories
        self.directory = directory
        self.transform = transform
        self._refresh()


    def __len__(self):
        return len(self.annotations)


    def __getitem__(self, idx):
        ann = self.annotations[idx]
        image = cv2.imread(ann['image_path'], cv2.IMREAD_COLOR)
        image = PIL.Image.fromarray(image)
        if self.transform is not None:
            image = self.transform(image)
        return image, ann['category_index']


    def _refresh(self):
        self.annotations = []
        for category in self.categories:
            category_index = self.categories.index(category)
            for image_path in glob.glob(os.path.join(self.directory, category, '*.jpg')):
                self.annotations += [{
                    'image_path': image_path,
                    'category_index': category_index,
                    'category': category
                }]

    def save_entry(self, image, category):
        """Saves an image in BGR8 format to dataset for category"""
        if category not in self.categories:
            raise KeyError('There is no category named %s in this dataset.' % category)

        filename = str(uuid.uuid1()) + '.jpg'
        category_directory = os.path.join(self.directory, category)

        if not os.path.exists(category_directory):
            subprocess.call(['mkdir', '-p', category_directory])

        image_path = os.path.join(category_directory, filename)
        cv2.imwrite(image_path, image)
        self._refresh()
        return image_path

    def get_count(self, category):
        i = 0
        for a in self.annotations:
            if a['category'] == category:
                i += 1
        return i

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('datasets.py','wb').write(src)

In [ ]:
!mkdir data
!mv data.zip data
!cd data;unzip data.zip

Archive:  data.zip
   creating: .ipynb_checkpoints/
   creating: Charlock/
   creating: Priority/
   creating: Stop/
   creating: Sugarbeet/
  inflating: Sugarbeet/07a8c624-ed9c-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/22949f94-ed9c-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/2617df1e-ed9c-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/2c77e26e-ed9c-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/fa8c40ec-ed9b-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/12642db0-ed9c-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/17e73520-ed9c-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/131b353c-ed9c-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/0f10e6c6-ed9c-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/322cc4e0-ed9c-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/26795cb2-ed9c-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/231a85d2-ed9c-11ed-94a5-a46bb6069316.jpg  
  inflating: Sugarbeet/31ead9f4-ed9c-11ed-94a5-a46bb6069316.jpg  
 

In [ ]:
import torchvision.transforms as transforms
# from dataset import ImageClassificationDataset

TASK = 'classification'
# TASK = 'emotions'
# TASK = 'fingers'
# TASK = 'diy'

CATEGORIES = ['Sugarbeet', 'Charlock', 'Stop', 'Priority']
# CATEGORIES = ['none', 'happy', 'sad', 'angry']
# CATEGORIES = ['1', '2', '3', '4', '5']
# CATEGORIES = [ 'diy_1', 'diy_2', 'diy_3']

DATASETS = ['A', 'B']
# DATASETS = ['A', 'B', 'C', 'D']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
# for name in DATASETS:
dataset = ImageClassificationDataset('data/', CATEGORIES, TRANSFORMS)

print("{} task with {} categories defined".format(TASK, CATEGORIES))

classification task with ['Sugarbeet', 'Charlock', 'Stop', 'Priority'] categories defined


In [ ]:
dataset.get_count('Charlock')

248

In [ ]:
# Set up the data directory location if not there already
# DATA_DIR = '/home/jetson/data/classification/'
# !echo "jetson" | sudo mkdir -p {DATA_DIR}

[sudo] password for jetson: 


### Data Collection
Execute the cell below to create the data collection tool widget. This cell should only take a few seconds to execute.

In [ ]:
!git clone https://github.com/NVIDIA-AI-IOT/jetcam
!cd jetcam; ls; python setup.py install

Cloning into 'jetcam'...
remote: Enumerating objects: 191, done.
remote: Total 191 (delta 0), reused 0 (delta 0), pack-reused 191
Receiving objects: 100% (191/191), 41.34 KiB | 13.78 MiB/s, done.
Resolving deltas: 100% (78/78), done.
jetcam	LICENSE.md  notebooks  README.md  setup.py
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: eas

In [ ]:
import cv2


def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
import ipywidgets
import traitlets
from IPython.display import display
# from jetcam.utils import bgr8_to_jpeg

# initialize active dataset
# dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')
save_widget = ipywidgets.Button(description='add')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# save image for category and update counts
def save(c):
    dataset.save_entry(camera.value, category_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
save_widget.on_click(save)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]), dataset_widget, category_widget, count_widget, save_widget
])

# display(data_collection_widget)
print("data_collection_widget created")

NameError: ignored

### Model
Execute the following cell to define the neural network and adjust the fully connected layer (`fc`) to match the outputs required for the project.  This cell may take several seconds to execute.

In [ ]:
import torch
import torchvision
import ipywidgets

device = torch.device('cuda')

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, len(dataset.categories))

# SQUEEZENET
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, len(dataset.categories), kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, len(dataset.categories))

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, len(dataset.categories))

model = model.to(device)

model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value='mmodel.pth')

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)

def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])

# display(model_widget)
print("model configured and model_widget created")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


model configured and model_widget created


### Live  Execution
Execute the cell below to set up the live execution widget.  This cell should only take a few seconds to execute.

In [ ]:
import threading
import time
from utils import preprocess
import torch.nn.functional as F

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.Text(description='prediction')
score_widgets = []
for category in dataset.categories:
    score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
    score_widgets.append(score_widget)

def live(state_widget, model, camera, prediction_widget, score_widget):
    global dataset
    while state_widget.value == 'live':
        image = camera.value
        preprocessed = preprocess(image)
        output = model(preprocessed)
        output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
        category_index = output.argmax()
        prediction_widget.value = dataset.categories[category_index]
        for i, score in enumerate(list(output)):
            score_widgets[i].value = score

def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget, score_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    ipywidgets.HBox(score_widgets),
    prediction_widget,
    state_widget
])

# display(live_execution_widget)
print("live_execution_widget created")

live_execution_widget created


### Training and Evaluation
Execute the following cell to define the trainer, and the widget to control it. This cell may take several seconds to execute.

In [ ]:
import torch.nn.functional as F
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
accuracy_widget = ipywidgets.FloatText(description='accuracy')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget

    # try:
    train_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True
    )

    # state_widget.value = 'stop'
    train_button.disabled = True
    eval_button.disabled = True
    time.sleep(1)

    if is_training:
        model = model.train()
    else:
        model = model.eval()
    while epochs_widget.value > 0:
        i = 0
        sum_loss = 0.0
        error_count = 0.0
        for images, labels in iter(train_loader):
            # send data to device
            images = images.to(device)
            labels = labels.to(device)

            if is_training:
                # zero gradients of parameters
                optimizer.zero_grad()

            # execute model to get outputs
            outputs = model(images)

            # compute loss
            loss = F.cross_entropy(outputs, labels)

            if is_training:
                # run backpropogation to accumulate gradients
                loss.backward()

                # step optimizer to adjust parameters
                optimizer.step()

            # increment progress
            error_count += len(torch.nonzero(outputs.argmax(1) - labels).flatten())
            count = len(labels.flatten())
            i += count
            sum_loss += float(loss)
            progress_widget.value = i / len(dataset)
            loss_widget.value = sum_loss / i
            accuracy_widget.value = 1.0 - error_count / i

        if is_training:
            epochs_widget.value = epochs_widget.value - 1
        else:
            break
    # except:
    #     print("error")
    #     pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    # state_widget.value = 'live'

train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))

train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    accuracy_widget,
    ipywidgets.HBox([train_button, eval_button])
])

# display(train_eval_widget)
print("trainer configured and train_eval_widget created")

trainer configured and train_eval_widget created


### Display the Interactive Tool!

The interactive tool includes widgets for data collection, training, and testing.

Execute the cell below to create and display the full interactive widget.

In [ ]:
import time
# Combine all the widgets into one display
all_widget = ipywidgets.VBox([
    # ipywidgets.HBox([train_eval_widget]),
    train_eval_widget,
    model_widget
])

display(all_widget)

In [ ]:
import shutil
from shutil import make_archive
directory = '/home/jetson/data/classification/classification_B'
shutil.make_archive('classb', 'zip', directory)

'/home/jetson/classification/classb.zip'

<h1 style="background-color:#76b900;"></h1>